# Getting Started with Db2 and Jupyter Notebooks

Jupyter notebooks include the ability to extend the syntax available within code blocks with a feature called Magic commands. Magic commands start with a percent sign `%` and provide a variety of features within a notebook environment, including modifying notebook behavior, executing OS commands, extending notebook functionality and with Db2 magic, a way of interacting with a Db2 database.

## Prerequisites
In order to use Db2 magic commands in your Jupyter notebook environment, you must have following prerequisites:
* Jupyter notebooks with a Python 3+ interpreter
* IBM Python driver (ibm_db)
* Db2 LUW access
* Optional: itables, pandas, multi-threading, matplotlib

To make sure that we have everything installed, the following code will load the libraries required need to run the Db2 magic commands.

In [ ]:
!pip3 install pandas multitasking ibm_db matplotlib itables

The first step to get up and running with Db2 Magic commands is to download the code to the directory that your Jupyter notebooks are stored in, or it can be downloaded directly from within a Jupyter notebook. The command format is shown below.

```
!wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb -O db2.ipynb
```

Start by running the next command to load the Db2 magic commands into your system.

In [ ]:
!wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb -O db2.ipynb

## Loading the Db2 Magic command extensions

The Db2 Magic commands rely on a Jupyter notebook extension, commonly refer to as a "magic" command. The beginning of all of the notebooks begin with the following command which will load the extension and allow the remainder of the notebook to use the `%sql` magic command.
<pre>
&#37;run db2.ipynb
</pre>
The cell below will load the Db2 extension. Note that it will take a few seconds for the extension to load, so you should generally wait until the "Db2 Extensions Loaded" message is displayed in your notebook. 

In [ ]:
%run db2.ipynb

Depending what version of Python you are running, the Pandas software may not support Db2 typing which will slow down some data type conversions.

# Connecting to Db2

Before any SQL commands can be issued, a connection needs to be made to the Db2 database that you will be using. The connection can be done manually (through the use of the CONNECT command), or automatically when the first `%sql` command is issued.

The Db2 magic command tracks whether a connection has occurred in the past and saves this information between notebooks and sessions. When you start up a notebook and issue a command, the program will reconnect to the database using your credentials from the last session. In the event that you have not connected before, the system will require that you execute the `CONNECT` command. 

The format of the `CONNECT` command is:
```
%sql CONNECT TO database USER userid USING password | ? 
                HOST ip_address PORT port_number 
                [ SSL ] [ SSLFILE file ] [ APIKEY key ] [ ACCESSTOKEN token ] 
                [ DNS "string" ]
```

The required fields are:

  * Database - Database name you want to connect to
  * Hostname - `localhost` if Db2 is running on your own machine or the IP address or symbolic name of the server
  * PORT - The port to use for connecting to Db2. This is usually 50000. 
  * USER - The user to connect as
  * USING - The password for the USER

You may also require additional pieces of information:

  * SSL Security - Request that SSL connections be used with Db2 and optionally the location of the SSL file that contains required certificate information
  * SSL File - The certificate file (if required)
  * API Key - An API key that is provided to connect to Db2
  * Access Token - An access token that is provided to connect to Db2
  * DSN - A string that contains the ODBC connection string 

The information supplied will be stored in the directory that the notebooks are stored on. Once you have entered the information, the system will attempt to connect to the database for you, and then you can run all SQL scripts. 

## Initial Connection

You must explicitly connect to a Db2 database for the first time. Subsequent connections are **not** required if you are connecting to the same database. When the Db2 magic command executes your SQL command, it checks to see whether you have already connected to the database. If not, it will check to see if there was a successful connection previously and will use those credentials to reconnect. The next statement will connect to the `SAMPLE` database on the Db2 server found at `10.0.0.1`.

In [ ]:
%sql connect to sample user db2inst1 using db2inst1 host 10.0.0.1 port 50000

### Closing and Resetting a Connection

`CONNECT CLOSE` will close the current connection, but will not reset the database parameters. This means that if you issue the `CONNECT` command again, the system will reconnect you to the database.
```
%sql CONNECT CLOSE
```

Let's close the current connection.

In [ ]:
%sql connect close

You can reconnect to the database by issuing the `CONNECT` command without any parameters. The system will use the previous connection information to connect to Db2.

In [ ]:
%sql connect

If you issue a SQL statement after closing the connection, the program will reconnect to last database you were connected to. For instance, we will close the connection and then select a count of EMPLOYEE records in the SAMPLE database.

In [ ]:
%sql connect close
%sql select count(*) from employee

A successful connection will always display the name of the database and the host that you connected to. If you don't want automatic connections to occur, make sure to use the `CONNECT RESET` command.

`CONNECT RESET` will close the current connection and remove any information on the connection. You will need to issue a new `CONNECT `statement with the connection information. 
```
%sql CONNECT RESET
```

Issuing a SQL statement after a `CONNECT RESET` will result in a connection error. 

In [ ]:
%sql connect reset
%sql select count(*) from employee

## Passwords

If you use a `?` for the password field, the system will prompt you for a password. This avoids typing the password as clear text on the screen. If a connection is not successful, the system will print the error message associated with the connect request.

Run the following SQL CONNECT command and user `db2inst1` as the password.

In [ ]:
%sql connect reset
%sql connect to sample user db2inst1 using ? host 10.0.0.1 port 50000

## Debugging Connection Strings

If you need to see the connection string that is generated by the `CONNECT` command, use the `-e` flag as part of the connection:
```
%sql -e connect to sample user db2inst1 using db2inst1
```

The program will **not** attempt to connect to the database, but will print the connection string that it was going to use. You can then modify this string to include any optional settings you require and then use the `DSN` option on the connect command to force the DSN value to be used instead of what the Db2 Magic command generates.


In [ ]:
%sql -e connect to sample user db2inst1 using db2inst1 host 10.0.0.1 port 50000

Copy the output from the previous command and use the following syntax to connect to the database by using the DSN keyword. This option provides you the ability to add additional keywords to the connection string to adjust the behavior of the ODBC driver.
```sql
%sql CONNECT DSN "dsn string"
```

In [ ]:
%sql CONNECT DSN "DRIVER={IBM DB2 ODBC DRIVER};DATABASE=SAMPLE;HOSTNAME=10.0.0.1;PORT=50000;UID=db2inst1;PWD=db2inst1;PROTOCOL=TCPIP;ConnectTimeout=15;loginTimeout=15;LONGDATACOMPAT=1;"

The connection details are not returned with this example.

## Connection Variable

The `CREDENTIALS` option allows you to use credentials that are stored in a variable. If your connection is successful, the variable will be saved to disk for future use. If you create another notebook and use the identical syntax, if the variable is not defined, the contents on disk will be used as the credentials. You should assign the credentials to a variable that represents the database (or schema) that you are communicating with. Using familiar names makes it easier to remember the credentials when connecting. 

Credentials must be supplied as a JSON dictionary that contains the following information:
```
{
  "database"    : "bludb",
  "hostname"    : "somesite-txn-blue.net",
  "port"        : 50000,
  "pwd"         : "somethingrandom",
  "uid"         : "myuserid",
  "ssl"         : "SSL",
  "sslfile"     : "filename",
  "apikey"      : "apikey",
  "accesstoken" : "accesstoken",
  "dsn"         : "dsn"
}
```
Some of these fields are not necessary. For instance, the apikey, accesstoken, and dsn fields are only required if you are using these forms of connections.

To use this as a connection document, assign this dictionary value to a variable:
```json
sample = {
  "hostname": "somesite-txn-blue.net",
  "pwd": "somethingrandom",...
}
```

Then use the following syntax to connect to the database:

```sql
%sql CONNECT CREDENTIALS sample
```

Create a connection variable for the `SAMPLE` database.

In [ ]:
sample = {
    "database" : "sample",
    "hostname" : "10.0.0.1",
    "port"     : "50000",
    "uid"      : "db2inst1",
    "pwd"      : "db2inst1"
}

Then use the `%sql CONNECT CREDENTIALS sample` command to connect to the `SAMPLE` database.

In [ ]:
%sql connect credentials sample

The system will attempt to connect to the database using the following process:

  1. If the `sample` variable exists in your Jupyter notebook, it will use the contents of that variable to connect to the database.
  2. If `sample` does not exist, the program will check the local directory that the notebook is in to find a file called `sample.pickle` which contains the credentials.
  3. If neither of the above exist the program will issue a connection error.

If a connection was successfully made, the credentials are written to disk using the name of the variable as the file name and the file type being `pickle`. A `pickle` file is a Python format that allows structures and data types to be written to a file and easily read back in. 

Subsequent notebooks in this lab will use the SAMPLE credentials to connect to the database. 

In [ ]:
%sql CONNECT RESET

#### Credits: IBM 2024, George Baklarz [baklarz@ca.ibm.com]